# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /home/ian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table(table_name = 'InsertTableName', con=engine)

X = df.message
Y = df[['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity',
'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report', ]]

In [4]:
df

message  \
0      a lot and we don't have any news of him. We ha...   
1      Since last night i cannot get through ( on cel...   
2      you have been said that you ganna put 100 gour...   
3                     How many aftershocks we can have.    
4      Haiti honey you're the rose thorns find the fi...   
5      Caution to take about the cyclone in Haiti spe...   
6                   Call me when you get this message.     
7                    information about season's cyclon.    
8      Digiceel! I'd like to know if will have seime ...   
9             Zb rmd : dl c.lve dl 2, dl rep, dl chase.    
10     We have a baby that died and another that has ...   
11     NOTES: Do not care about this message because ...   
12     Good morning i need some information i never g...   
13     May i know if the schools will find help for t...   
14     Hallo president if the requested i ask you can...   
15     NOTES: It's not necessary to translate this me...   
16     NOTES: Do not care about this message because ...   
17     Hello, even if I have not answered the questio...   
18     Greetings to all the people receiving those te...   
19     Until now I never found anything of the aid, t...   
20     Jouna you should take precautions this afterno...   
21     please Do education ministry started booklet p...   
22     I would like to have more informations about y...   
23     yet to follow that advice during the hurricane...   
24     I need help please because my father died, my ...   
25     NOTES: This message message is not important, ...   
26     NOTES: I can not translate the message because...   
27     I would like to know if the faculties will open.    
28     If anyone is qualified in a profession,can you...   
29             Digicel, the company without limitation.    
...                                                  ...   
26185  You do not pass to see me at all what I have d...   
26186                   Sunday 14 Ysemne muri ms 2o1o.     
26187           NOTES: this is not an important message.   
26188  Hello my friend, You're OK! I wish you good ni...   
26189  We are a committee that works to help the vict...   
26190  NOTES: This message is nonsensical. It's not n...   
26191  Hello my friend, cmt did you spend the day?. P...   
26192  we got a big problem to feed in 22 area w'r ju...   
26193  A man without a job is a wolf without teeth, l...   
26194  Strive to end so you seem to not my one such 2...   
26195  Li (u)  3! Sodium (Na)  11! Potasium (k)  19! ...   
26196  You know that the water turns to ice and flowe...   
26197  I am from Morne Lazarre, now I am in Morne Cal...   
26198         We need help please my family .Good noght    
26199                   Zb rmd : dl 2, dl rep, dl chase    
26200  We are hungry, thirsty, have always had to sle...   
26201                    I'd like to know what 4636 is?    
26202  4636 I NEED AN ANSWER ON MY SUBJECT,TELL ME WH...   
26203  NOTES: It is not necessary to translate this m...   
26204  NOTES: Do not care about this message. It's an...   
26205                NOTES: This message is nonsensical.   
26206                 Is the earthquake possible again.    
26207  NOTES: this person is playing the message is n...   
26208  I would like to make a death note,where can i ...   
26209  NOTES: It's not an important message for my work.   
26210     I would like more information on the country.    
26211          Why do not you send me any message back.    
26212  Hello everybody I'd like to know the complete ...   
26213  NOTES: The person joking, the message is not i...   
26214  Good evening! when do you preview the next ear...   

                                                original   genre  related  \
0      anpil E nou pa gen oken nouvel li, menm ke a n...  direct        1   
1      De pi ye swa m pa ka pase. Yo toujou di mwen r...  direct        0   
2      Nou te di n'ap mete 100 gdes sou telefon nan p...  direct        0   
3                   

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
rf = RandomForestClassifier()

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator = rf)),#('clf', OneVsRestClassifier(MultiOutputClassifier(estimator = rf))),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [43]:
y_pred.shape[1]

36

In [42]:
y_test.shape

(6554, 36)

In [13]:
# evaluate all steps on test set
y_pred = pipeline.predict(X_test)

for col in range(y_pred.shape[1]):
    print("Class: " + str(col))
    print(classification_report(y_test.iloc[:, col], y_pred[:, col]))

Class: 0
             precision    recall  f1-score   support

          0       0.61      0.37      0.46      1477
          1       0.83      0.93      0.88      5027
          2       0.70      0.14      0.23        50

avg / total       0.78      0.80      0.78      6554

Class: 1
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5471
          1       0.82      0.38      0.52      1083

avg / total       0.88      0.88      0.86      6554

Class: 2
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6516
          1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

Class: 3
             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3810
          1       0.74      0.52      0.61      2744

avg / total       0.73      0.72      0.71      6554

Class: 4
             precision    recall  f1-score

/home/ian/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2))
        }
        #,
'''
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
        
    }'''

cv =  GridSearchCV(pipeline, param_grid=parameters)


'''
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
'''

"\n    parameters = {\n        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),\n        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),\n        'features__text_pipeline__vect__max_features': (None, 5000, 10000),\n        'features__text_pipeline__tfidf__use_idf': (True, False),\n        'clf__n_estimators': [50, 100, 200],\n        'clf__min_samples_split': [2, 3, 4],\n        'features__transformer_weights': (\n            {'text_pipeline': 1, 'starting_verb': 0.5},\n            {'text_pipeline': 0.5, 'starting_verb': 1},\n            {'text_pipeline': 0.8, 'starting_verb': 1},\n        )\n    }\n\n    cv = GridSearchCV(pipeline, param_grid=parameters)\n"

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

Accuracy = TP+TN/TP+FP+FN+TN

Precision = TP/TP+FP

Recall = TP/TP+FN



In [32]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()
    precision = (y_pred == y_test).sum() / len(y_pred)
    recall = (y_pred == y_test).sum() / y_pred.sum()
    
    print("Labels:", labels)

    print("Accuracy:")
    print(accuracy)
    
    print("Precision:")
    print(precision)
    
    print("Recall:")
    print(recall)
    
    
    #print("\nBest Parameters:", cv.best_params)

    
display_results(cv, y_test, y_pred)

Labels: [0 1 2]
Accuracy:
related                   0.797833
request                   0.883430
offer                     0.994202
aid_related               0.724596
medical_help              0.918828
medical_products          0.952853
search_and_rescue         0.972078
security                  0.981385
military                  0.968111
child_alone               1.000000
water                     0.946445
food                      0.913488
shelter                   0.928441
clothing                  0.986421
money                     0.978486
missing_people            0.987183
refugees                  0.965212
death                     0.959262
other_aid                 0.874733
infrastructure_related    0.932560
transport                 0.954684
buildings                 0.949191
electricity               0.977876
tools                     0.994355
hospitals                 0.987336
shops                     0.997101
aid_centers               0.988404
other_infrastructure      0.9

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.